In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import warnings
# PACF
import  statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.lines import lineStyles
from numpy.ma.extras import corrcoef
from pandas.tseries.holiday import holiday_calendars

pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
stores = pd.read_csv('stores.csv')
transactions = pd.read_csv('transactions.csv').sort_values(['store_nbr', 'date'])
oil_price =pd.read_csv('oil.csv')

train['date'] = pd.to_datetime(train.date)
test['date'] = pd.to_datetime(test.date)
transactions['date'] = pd.to_datetime(transactions.date)

#

In [ ]:
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype('float32')
stores.cluster = stores.cluster.astype('int8')

train.head()

In [ ]:
temp = pd.merge(train.groupby(["date", "store_nbr"]).sales.sum().reset_index(), transactions, how = "left")
print("Spearman Correlation between Total Sales and Transactions: {:,.4f}".format(temp[['sales', 'transactions']].corr(method = "spearman").sales.loc["transactions",]))
px.line(transactions.sort_values(["store_nbr", "date"]), x = 'date', y='transactions', color = 'store_nbr', title = "Transactions")

In [ ]:
a = transactions.copy()


a["year"] = a.date.dt.year
a["month"] = a.date.dt.month
px.box(a, x="year", y="transactions" , color = "month", title = "Transactions")

In [ ]:
a = transactions.set_index('date').resample("M").transactions.mean().reset_index()
a['year'] = a.date.dt.year
px.line(a, x = 'date', y='transactions', color = 'year', title = 'Monthly Average Transactions')

In [ ]:
a

In [ ]:
px.scatter(temp, x = 'transactions', y = 'sales', trendline = 'ols', trendline_color_override = 'red')

In [ ]:
a=transactions.copy()
a['year'] = a.date.dt.year
a['dayofweek'] = a.date.dt.dayofweek + 1
a = a.groupby(["year", "dayofweek"]).transactions.mean().reset_index()

px.line(a,x = 'dayofweek', y = 'transactions', color = 'year', title = 'Transactions')

In [ ]:
oil = (pd.read_csv("oil.csv"))

oil['date'] = pd.to_datetime(oil.date)
oil = oil.set_index('date').dcoilwtico.resample("D").sum().reset_index()

oil['dcoilwtico'] = np.where(oil['dcoilwtico'] == 0, np.nan, oil['dcoilwtico'])
oil['dcoilwtico_interpolated'] = oil.dcoilwtico.interpolate()

p = oil.melt(id_vars = ['date'] + list(oil.keys()[5:]), var_name = 'Legend')
px.line(p.sort_values(['Legend', 'date'], ascending = [False, True]), x = 'date', y = 'value', color = 'Legend', title = 'Daily Oil Price')

In [ ]:
temp = pd.merge(temp, oil, how = 'left')
print("Correlation with Daily Oil Prices")
print(temp.drop(["store_nbr", "dcoilwtico"], axis = 1).corr("spearman").dcoilwtico_interpolated.loc[["sales", "transactions"]], "\n")

fig, axes = plt.subplots(1, 2, figsize = (15, 5))
temp.plot.scatter(x = "dcoilwtico_interpolated", y = "transactions", ax = axes[0])
temp.plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[1], color = 'red')

axes[0].set_title("Daily oil price & Transactions", fontsize = 15)
axes[1].set_title("Daily oil price & Sales", fontsize = 15)

In [ ]:
a = pd.merge(train.groupby(['date', 'family']).sales.sum().reset_index(), oil.drop('dcoilwtico', axis = 1), how = 'left')

In [ ]:
c = a.groupby('family').corr("spearman").reset_index()
c = c[c.level_1 == "dcoilwtico_interpolated"][["family", 'sales']].sort_values('sales')

fig, axes = plt.subplots(7, 5, figsize = (20, 20))
for i, fam in enumerate(c.family):
    if i < 6:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y ='sales', ax = axes[0, i-1])

        axes[0, i-1].axvline(x = 70, color = 'r', linestyle = "--")

        axes[0, i - 1].set_title(fam + '\n Correlation:' + str(c[c.family == fam].sales.iloc[0])[:6], fontsize = 12)

    if i >= 6 and i < 11:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[1, i - 6])

        axes[1, i-6].set_title(fam + "\n Correlation" + str(c[c.family == fam].sales.iloc[0])[:6],
                                   fontsize = 12)

        axes[1, i-6].axvline(x=70, color = 'r', linestyle = '--')
    if i >= 11 and i < 16:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[2, i - 11])

        axes[2, i - 11].set_title(fam + '\n Correlations:' + str(c[c.family == fam].sales.iloc[0])[:6],
                                                                 fontsize = 12)
        axes[2, i - 11].axvline(x = 70, color = 'r', linestyle = '--')
    if i >= 16 and i < 21:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[3, i - 16])
        axes[3, i - 16].set_title(fam + '\n Correlation:' + str(c[c.family == fam].sales.iloc[0])[:6], fontsize = 12)
        axes[3, i - 16].axvline(x = 70, color = 'r', linestyle = '--')
    if i >= 21 and i < 26:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[4, i - 21])
        axes[4, i - 21].set_title(fam + '\n Correlation' + str(c[c.family == fam].sales.iloc[0])[:6], fontsize = 12)
        axes[4, i-21].axvline(x = 70, color = 'r', linestyle = '--')
    if i >= 26 and i < 31:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[5, i - 26])
        axes[5, i-26].set_title(fam + '\n Correlation' + str(c[c.family == fam].sales.iloc[0])[:6], fontsize = 12)
        axes[5, i-26].axvline(x = 70, color = 'r', linestyle = '--')
    if i >= 31:
        a[a.family == fam].plot.scatter(x = 'dcoilwtico_interpolated', y = 'sales', ax = axes[6, i-31])
        axes[6, i-31].set_title(fam + '\n Correlation:' + str(c[c.family == fam].sales.iloc[0])[:6], fontsize = 12)
plt.tight_layout(pad = 5)
plt.suptitle("Daily Oil Product & Total Family Sales \n", fontsize = 20)
plt.show()

In [ ]:
a = train[["store_nbr", "sales"]]

a['ind'] =1
a["ind"] = a.groupby("store_nbr").ind.cumsum().values

a = pd.pivot(a, index = 'ind', columns = 'store_nbr', values = 'sales').corr()
mask = np.triu(a.corr())

plt.figure(figsize = (20, 20 ))
sns.heatmap(a,
           annot = True,
           fmt = '.1f',
           cmap = 'coolwarm',
           square = True,
           mask = mask,
           linewidth = 1,
           cbar = False)
plt.title("Correlation among stores", fontsize = 20)
plt.show()

In [ ]:
a = train.set_index("date").groupby("store_nbr").resample("D").sales.sum().reset_index()
px.line(a, x='date', y = 'sales', color = 'store_nbr', title = "Daily total sales of the stores")

In [ ]:
train = train[~((train.store_nbr == 52) & (train.date <'2017-04-20' ))]
train = train[~((train.store_nbr == 22) & (train.date < '2015-10-09'))]
train = train[~((train.store_nbr== 42) & (train.date < '2015-08-21'))]
train = train[~((train.store_nbr== 21) & (train.date < '2015-07-24'))]
train = train[~((train.store_nbr== 29) & (train.date < '2015-03-20'))]
train = train[~((train.store_nbr==20) & (train.date < '2015-02-13'))]
train = train[~((train.store_nbr == 53) & (train.date < '2014-05-29'))]
train = train[~((train.store_nbr == 36) & (train.date < '2013-05-09'))]
train.shape

In [ ]:
c = train.groupby(['store_nbr', 'family']).sales.sum().reset_index().sort_values(['family','store_nbr'])
c = c[c.sales == 0]
c

In [ ]:
print(train.shape)
# Anti join
outer_join = train.merge(c[c.sales == 0].drop('sales', axis = 1), how = 'outer', indicator = True)
train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
del outer_join
gc.collect()
train.shape

In [ ]:
zero_prediction = []
for i in range(0, len(c)):
    zero_prediction.append(
        pd.DataFrame({
            "date": pd.date_range("2017-08-16", "2017-08-31").tolist(),
            "store_nbr": c.store_nbr.iloc[i],
            "family":c.family.iloc[i],
            "sales":0

        })
    )
zero_prediction = pd.concat(zero_prediction)
del c
gc.collect()
zero_prediction

In [ ]:
c = train.groupby(["family", "store_nbr"]).tail(60).groupby(["family", "store_nbr"]).sales.sum().reset_index()
c[c.sales == 0]

In [ ]:
fig, ax = plt.subplots(1, 5, figsize = (20, 4))
train[(train.store_nbr == 10) & (train.family == "LAWN AND GARDEN")].set_index("date").sales.plot(ax = ax[0], title = "STORE 10 - LAWN AND GARDEN")

train[(train.store_nbr == 36) & (train.family == "LADIESWEAR")].set_index('date').sales.plot(ax = ax[1], title = "STORE 36 - LADIESWEAR")

train[(train.store_nbr == 6) & (train.family == "SCHOOL AND OFFICE SUPPLIES")].set_index('date').sales.plot(ax = ax[2], title = "STORE 6 - SCHOOL AND OFFICE SUPLIES")

train[(train.store_nbr == 14) & (train.family =='BABY CARE')].set_index('date').sales.plot(ax = ax[3], title = "STORE 14 - BABY CARE")

train[(train.store_nbr== 42) & (train.family == 'BOOKS')].set_index('date').sales.plot(ax = ax[4], title = "STORE 42 - BOOKS")


In [ ]:
a = train.set_index('date').groupby("family").resample("D").sales.sum().reset_index()
px.line(a, x = 'date', y = 'sales', color = 'family', title = "Daily total sales of the family")

In [ ]:
a = train.groupby('family').sales.mean().sort_values(ascending = False).reset_index()
px.bar(a, y = 'family', x = 'sales', color = "family", title = "Which product family preferred more?")


In [ ]:
d = pd.merge(train, stores)
d['store_nbr'] = d['store_nbr'].astype('int8')
d['year'] = d.date.dt.year
px.line(d.groupby(["city", "year"]).sales.mean().reset_index(), x = 'year', y = 'sales', color = 'city')

In [ ]:
import pandas as pd

holidays = pd.read_csv('holidays_events.csv')
holidays['date'] = pd.to_datetime(holidays.date)

# Extract transferred holidays (original holidays that were moved)
tr1 = holidays[(holidays.type == 'Holiday') & (holidays.transferred == True)].drop("transferred", axis=1)

# Extract transfer days (new dates replacing the transferred holidays)
tr2 = holidays[(holidays.type == 'Transfer')].drop('transferred', axis=1)

# Combine both into one DataFrame
tr = pd.concat([tr1, tr2], axis=0).reset_index(drop=True)

# Filter out the transferred and transfer types from original holidays
holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop('transferred', axis=1)

# Append the transferred holidays and their transfer dates back
holidays = pd.concat([holidays, tr], axis=0).reset_index(drop=True)

